In [1]:
import torch
print("Is GPU available?:", torch.cuda.is_available())
print("CUDA Device Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")

Is GPU available?: True
CUDA Device Name: Tesla T4


In [2]:
import torch
print("PyTorch version:", torch.__version__)
print("CUDA version:", torch.version.cuda)

PyTorch version: 2.4.0
CUDA version: 12.3


In [3]:
#Loading data
import pandas as pd

#Loading the data from a CSV file
df1 = pd.read_csv("/kaggle/input/dataset/cicddos2019_dataset.csv")
print("Data Loaded Successfully!")

Data Loaded Successfully!


In [4]:
#Working on the copy of original dataset
df=df1
df.head()

,Unnamed: 0,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Class
0,0,17,216631,6,0,2088.0,0.0,393.0,321.0,348.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UDP,Attack
1,1,17,2,2,0,802.0,0.0,401.0,401.0,401.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UDP,Attack
2,2,17,48,2,0,766.0,0.0,383.0,383.0,383.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UDP,Attack
3,3,17,107319,4,0,1398.0,0.0,369.0,330.0,349.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UDP,Attack
4,4,17,107271,4,0,1438.0,0.0,389.0,330.0,359.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UDP,Attack


In [5]:
!pip install sdv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.0/154.0 kB 4.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 94.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: platformdirs
    Found existing installation: platformdirs 3.11.0
    Uninstalling platformdirs-3.11.0:
      Successfully uninstalled platformdirs-3.11.0
  Attempting uninstall: botocore
    Found existing installation: botocore 1.35.23
    Uninstalling botocore-1.35.23:
      Successfully uninstalled botocore-1.35.23
  Attempti

In [6]:
from sdv.metadata import Metadata

metadata = Metadata.detect_from_dataframe(
    data=df,
    table_name='CICDDoSAttack')

In [7]:
# save_to_json
metadata.save_to_json(filepath='my_metadata_v1.json')

#load_from_json
metadata = Metadata.load_from_json(filepath='my_metadata_v1.json')

In [8]:
from sdv.single_table import CTGANSynthesizer

# Creating a synthesizer
synthesizer = CTGANSynthesizer(metadata, epochs=100, batch_size=500, verbose=True, cuda=True)

In [9]:
# get_parameters
synthesizer.get_parameters()

{'enforce_min_max_values': True,
 'enforce_rounding': True,
 'locales': ['en_US'],
 'embedding_dim': 128,
 'generator_dim': (256, 256),
 'discriminator_dim': (256, 256),
 'generator_lr': 0.0002,
 'generator_decay': 1e-06,
 'discriminator_lr': 0.0002,
 'discriminator_decay': 1e-06,
 'batch_size': 500,
 'discriminator_steps': 1,
 'log_frequency': True,
 'verbose': True,
 'epochs': 100,
 'pac': 10,
 'cuda': True}

In [11]:
# get_metadata
metadata = synthesizer.get_metadata()

In [14]:
# Learning from your data
synthesizer.fit(df)

Gen. (-0.26) | Discrim. (-1.20): 100%|██████████| 100/100 [2:08:57<00:00, 77.37s/it] 


In [15]:
# get_loss_values
synthesizer.get_loss_values()

,Epoch,Generator Loss,Discriminator Loss
0,0,-1.679598,-0.202447
1,1,-0.952264,-0.491004
2,2,-0.010150,-1.161366
3,3,0.005432,-0.261983
4,4,-1.721356,-0.178633
...,...,...,...
95,95,-0.935172,-0.292681
96,96,-1.861829,-0.480013
97,97,-0.952131,-0.323237
98,98,-1.912066,-1.091144


In [22]:
# get_loss_values_plot
fig = synthesizer.get_loss_values_plot()
fig.show()

In [23]:
# Saving your synthesizer
synthesizer.save(
    filepath='my_synthesizer.pkl'
)
print("Saved Successfully!!!")

Saved Successfully!!!


In [24]:
# CTGANSynthesizer.load
from sdv.single_table import CTGANSynthesizer

synthesizer = CTGANSynthesizer.load(
    filepath='my_synthesizer.pkl'
)

/opt/conda/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



In [25]:
# Generate synthetic data using the trained synthesizer
synthetic_data = synthesizer.sample(len(df))

# Display the first few rows of the synthesized data
print(synthetic_data.head())

   Unnamed: 0  Protocol  Flow Duration  Total Fwd Packets  \
0      281335        17              1                199   
1       95434        17        2980687                  4   
2      306200         6              1                  4   
3       69820         6       37318940                 13   
4      102151         6              1                  1   

   Total Backward Packets  Fwd Packets Length Total  Bwd Packets Length Total  \
0                       0                   89381.0                      28.0   
1                       0                    2770.0                       0.0   
2                       0                       0.0                       0.0   
3                       6                     205.0                       0.0   
4                       2                     548.0                       0.0   

   Fwd Packet Length Max  Fwd Packet Length Min  Fwd Packet Length Mean  ...  \
0                  440.0                  440.0              442.4

In [27]:
# Save the synthetic data to a CSV file
synthetic_data.to_csv("Synthesized_Dataset.csv", index=False)
print("Synthetic dataset saved as 'Synthesized_Dataset.csv'")

Synthetic dataset saved as 'Synthesized_Dataset.csv'


In [29]:
# Get the shape of the synthesized dataset (rows, columns)
print("Shape of the synthesized dataset:", synthetic_data.shape)

Shape of the synthesized dataset: (431371, 80)
